In [1]:
from functools import partial
import glob
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
import pybedtools as pbt
import re
import subprocess
import sys

sys.path.append("/dors/capra_lab/users/fongsl/tools/py_/")
sys.path.append("/dors/capra_lab/users/fongsl/tools/genome/")
import config_readwrite as crw
import chr_functions
import split_filename

In [5]:
name = "/data/hodges_lab/ATAC-STARR_B-cells/bin_human-evolution/config"
config, configfile_name = crw.read_config(name)

SHUF_PATH = config["SHUFFLES"]["path"]
REGIONS = config["CIS_TRANS"]["regions"]

CL = "GM12878"
FP_PATH = config["TF_FOOTPRINTING_JASPAR"][f"{CL}_bindetect"]
FP_RE = config[f"TF_FOOTPRINTING_JASPAR_{CL}"]["FP"]
ID_TAG = config["TF_FOOTPRINTING_JASPAR"]["ID_TAG"]

RE = config["TF_FOOTPRINTING"]["results"]

path, region_file, region = split_filename.split_filename(REGIONS)

In [4]:
REGIONS

'/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/regions/all_uniq_diffAct_regions2.bed'

0. string split to get TF name
1. intersect regions w footprint files
2. create matrix w/ region_id (should be 17605 regions x 693 archetypes) 

In [4]:
a = "Isl1_MA1608.1"
a.upper()

'ISL1_MA1608.1'

# functions

## step 0 

In [5]:
def get_arch_id(filename):
    
    match = re.match(r"([a-z]+)([0-9]+)", filename.split("_")[0], re.I)  # get the archetype ID w/ chr/int split on first two values
    
    if match:
        items = match.groups()
    arch_id = ("".join(items))
    
    return arch_id

In [6]:
def get_tf_bound_file(f, filename, cell_line):

    tf_bound_file = os.path.join(f, "beds", f"{filename}_{cell_line}_1.8-filter_bound.bed")
    
    return tf_bound_file

In [12]:
def get_tf_paths(path, cell_line, id_tag):
    """
    return dictionary w/ key = archetype id and value = tuple of TF name and full file path.  
    
    1. get all the file names
    2. get arch_id name - string split function
    3. get TF name - str split
    4. get TF FP bound file
    5. make dictionary
    6. return dictionary
    """
    tf_dict ={}
    
    #1
    fs = glob.glob(os.path.join(path, f"*{id_tag}*"))
    print(len(fs))
    for f in fs:
        #2
        filename = os.path.split(f)[1]  # get the file name
        #3
        if "MA" in id_tag:
            arch_id = filename.upper()  
            tf_name = filename.split("_")[0]
        else:
            arch_id=get_arch_id(filename) #FOR ARCHTYPES ONLY
            
            #4
            tf_name = "_".join((filename.split(arch_id)[1]).split("_")[:-1])  # get TF name by splitting string. 

        
        #5
        tf_bound_file = get_tf_bound_file(f, filename, cell_line)  # get path to TF bound.bed file
        #6
        tf_dict[arch_id] = (tf_name, tf_bound_file)
    
    return tf_dict

## step 1 

In [8]:
def intersect_w_fp(regions, fp_name, fp_bed, outdir, cell_line):
    
    if os.path.exists(os.path.join(outdir, cell_line)) is False:  # make a directory
        os.mkdir(os.path.join(outdir, cell_line))
        
    out = os.path.join(outdir, cell_line, f"{cell_line}-{fp_name}.bed")
    
    if os.path.exists(out) is False:  # if intersection hasn't already been done. 
        
        bedregions = pbt.BedTool(regions)
    
        bedregions.intersect(pbt.BedTool(fp_bed), wo = True).saveas(out)
    
    return out
    

In [9]:
def make_region_fp_matrix(outdir, cell_line, id_tag):
    
    """
    return dictionary of sample_id keys and pandas dataframes (values) for overlapping TF footprints
    1. get all region_x_TF.bed intersection files
    2. get sample_id 
    3. pick which columns and column names to load from file
    4. load file
    5. FILTER for overlaps w/ >5bp (size of TFBS)
    6. add to dictionary
    7. return dictionary
    """
    #1
    fs = glob.glob(os.path.join(outdir, cell_line, f"*{id_tag}*.bed"))
    
    dfs = {}
    for i, f in enumerate(fs):
        
        #2
        path, file, sample_id = split_filename.split_filename(f)
        #4
        usecols = list(np.arange(8))
        usecols.append(21)
        
        colnames = ["#chr", "start", "end", 'region_id', 
                    "#chr_fp", "start_fp", "end_fp", 
                    "fp_id", "overlap"]
        #4
        d = pd.read_csv(f, sep = '\t', header = None, usecols = usecols, names = colnames)
        #5
        d = d.loc[d["overlap"]>=5]
        #6
                
        arch_id = (sample_id.split(".bed")[0]).split("-")[-1]
        d[arch_id] = True
        #7
        dfs[sample_id] = d[["region_id", arch_id]].drop_duplicates()
        
        print(i)
    return dfs

## step 2

In [10]:
def join_fp_matrices(dfs, region_file):
    
    """
    return pandas dataframe of all fp matrices
    1. open the region file
    2. consecutively join all other region_x_fp intersections
    3. fillin na w/ False
    
    """
    #1
    region_f = pd.read_csv(region_file, sep = '\t')
    
    #2
    for i, v in enumerate(dfs.values()):

        if i == 0:  # only merge the region file the first time
            regions = pd.merge(region_f, v, how = "left", on = "region_id")  
        else:
            regions = pd.merge(regions, v, how = "left")
    print(regions.shape)
    
    #3
    regions = regions.fillna(False)
    
    return regions

# main

In [11]:
FP_RE

'/data/hodges_lab/ATAC-STARR_B-cells/results/results_human-evolution/TF_footprinting/GM12878_1.8-JASPAR_0.05-pval_morethan5bp.bed'

In [13]:
CLS =["GM12878", "LCL8664"]
region_f = pd.read_csv(REGIONS, sep = '\t')
for CL in CLS:
    FP_PATH = config["TF_FOOTPRINTING_JASPAR"][f"{CL}_bindetect"]
    FP_RE = config["TF_FOOTPRINTING_JASPAR"][f"{CL}_FP"]

    if os.path.exists(FP_RE) is False:
        tfs = get_tf_paths(FP_PATH, CL, ID_TAG) ## get all the tf.bed paths

        for k, vs in tfs.items():  # per TF footprint file
            tfname, FP_bed = vs[0], vs[1]

            out = intersect_w_fp(REGIONS, k, FP_bed, RE, CL)  # do the intersections. 

        dfs = make_region_fp_matrix(RE, CL, ID_TAG)  # open the intersections as pandas dataframes, keep only FP overlaps > 5

        regions = join_fp_matrices(dfs, REGIONS)

        regions.to_csv(FP_RE, sep = '\t', index = False)
    else:
        print("done!")

746
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

# shuffle

In [14]:
shufs = glob.glob(os.path.join(SHUF_PATH, "shuf-all_uniq_diffAct_regions2-*.bed"))

In [15]:
CL = "GM12878"
FP_PATH = config["TF_FOOTPRINTING_JASPAR"][f"{CL}_bindetect"]

tfs = get_tf_paths(FP_PATH, CL, ID_TAG) ## get all the tf.bed paths
new_cls = []  # make alist of the new cl names for shuffles

for SHUF in shufs:
    
    path, region_file, region = split_filename.split_filename(SHUF)
 
    CL_SHUF = f"{region}-{CL}"
    print(CL_SHUF)
    for k, vs in tfs.items():  # per TF footprint file
        tfname, FP_bed = vs[0], vs[1]        
        out = intersect_w_fp(SHUF, k, FP_bed, RE, CL_SHUF)  # do the intersections. 

    dfs = make_region_fp_matrix(RE, CL_SHUF, ID_TAG)  # open the intersections as pandas dataframes, keep only FP overlaps > 5

    regions = join_fp_matrices(dfs, REGIONS)
    
    FP_RE = os.path.join(RE, f"{CL_SHUF}_1.8-JASPAR_0.05-pval_morethan5bp.bed")
    regions.to_csv(FP_RE, sep = '\t', index = False)

746
shuf-all_uniq_diffAct_regions2-5-GM12878
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
2

609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
(17604, 750)
shuf-all_uniq_diffAct_regions2-9-GM12878
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126


475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724


339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588


# differential footprints

## do this later. First just find out which FP are enriched in humans.  

In [ ]:
CLS =["GM12878", "LCL8664"]
for CL in CLS:
    FP_RE = config["TF_FOOTPRINTING"][f"{CL}_FP"]

reference this:
https://stackoverflow.com/questions/44578571/intersect-two-boolean-arrays-for-true